In [1]:
%run helper/setup_notebook.ipynb import display_table

Successfully connected to leetcode50 database.


In [2]:
display_table('Employee')

+-------+--------+------------+--------+
| empId |  name  | supervisor | salary |
+-------+--------+------------+--------+
|   3   |  Brad  |    None    |  4000  |
|   1   |  John  |     3      |  1000  |
|   2   |  Dan   |     3      |  2000  |
|   4   | Thomas |     3      |  4000  |
+-------+--------+------------+--------+


In [3]:
display_table('Bonus')

+-------+-------+
| empId | bonus |
+-------+-------+
|   2   |  500  |
|   4   |  2000 |
+-------+-------+


### Write an SQL query to report the name and bonus amount of each employee with a bonus less than 1000.

```
+------+-------+
| name | bonus |
+------+-------+
| Brad | null  |
| John | null  |
| Dan  | 500   |
+------+-------+
```

In [4]:
%%sql 

SELECT 
    name, bonus 
FROM employee e
LEFT JOIN bonus b ON e.empId = b.empId
WHERE bonus < 1000 OR bonus IS NULL

name,bonus
Brad,None
John,None
Dan,500


# Using Pandas

In [5]:
import pandas as pd 


In [6]:
employee_query = %sql SELECT * FROM employee # type: ignore
bonus_query = %sql SELECT * FROM bonus # type: ignore
employee_df = employee_query.DataFrame()
bonus_df = bonus_query.DataFrame()
display(employee_df, bonus_df)

,empId,name,supervisor,salary
0,3,Brad,NaN,4000
1,1,John,3.0,1000
2,2,Dan,3.0,2000
3,4,Thomas,3.0,4000


,empId,bonus
0,2,500
1,4,2000


## merge()

In [7]:
merged_df = employee_df.merge(bonus_df, on='empId', how='left')
merged_df

,empId,name,supervisor,salary,bonus
0,3,Brad,NaN,4000,NaN
1,1,John,3.0,1000,NaN
2,2,Dan,3.0,2000,500.0
3,4,Thomas,3.0,4000,2000.0


In [9]:
merged_df.loc[(merged_df['bonus'].isna()) | (merged_df['bonus'] < 1000)]

,empId,name,supervisor,salary,bonus
0,3,Brad,NaN,4000,NaN
1,1,John,3.0,1000,NaN
2,2,Dan,3.0,2000,500.0


In [12]:
merged_df.loc[(merged_df['bonus'].isna()) | (merged_df['bonus'] < 1000)] \
    [['name', 'bonus']]

,name,bonus
0,Brad,NaN
1,John,NaN
2,Dan,500.0


## query()

In [22]:
merged_df.query("bonus < 1000 or bonus.isnull()")

,empId,name,supervisor,salary,bonus
0,3,Brad,NaN,4000,NaN
1,1,John,3.0,1000,NaN
2,2,Dan,3.0,2000,500.0


In [24]:
merged_df.query("bonus < 1000 or bonus.isnull()") \
    [['name', 'bonus']]. \
    set_index('name', drop=True)

,bonus
name,
Brad,NaN
John,NaN
Dan,500.0


## join() and filter()

In [26]:
joined_df = employee_df.set_index('empId').join(bonus_df.set_index('empId'))
joined_df

,name,supervisor,salary,bonus
empId,,,,
3,Brad,NaN,4000,NaN
1,John,3.0,1000,NaN
2,Dan,3.0,2000,500.0
4,Thomas,3.0,4000,2000.0


In [30]:
joined_df.filter(items=['name', 'bonus']). \
    loc[(joined_df['bonus'].isna()) | (joined_df['bonus'] < 1000)]. \
    set_index('name', drop=True)

,bonus
name,
Brad,NaN
John,NaN
Dan,500.0
